In [1]:

import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from eegsc.preprocessing.filters import BandPassFilter
from eegsc.preprocessing.spectrum import SpectrumTransformer
from eegsc.utils.io import read_raw
from eegsc.utils.path import get_data_path
from eegsc.utils.experiments import create_spectrum_dataset
from eegsc.utils.cross_val import train_test_split, cross_val_score

In [2]:
start_time = 5
signal_duration = 10 - start_time
bandpass_filter = BandPassFilter(signal_duration=signal_duration)
spectrum_transformer = SpectrumTransformer(psd_method='periodogram',
                                           signal_duration=signal_duration,
                                           features_set='iaf_paf_psd_trp')

In [3]:
data = read_raw(os.path.join(get_data_path(), 'raw', '1st_Day.mat'),
                # data_types=['left_real', 'right_real',
                #             'left_im1', 'right_im1',
                #             'left_im2', 'right_im2',
                #             'left_quasi', 'right_quasi'])
                # data_types=['left_quasi', 'right_quasi'])
                # data_types=['left_im2', 'right_im2'])
                # data_types=['left_im1', 'right_im1'])
                data_types=['left_real', 'right_real'])

In [4]:
statistics, labels, person_idxs = create_spectrum_dataset(
    data, bandpass_filter, spectrum_transformer, start_time=start_time, is_len_equal=True)
statistics.shape, labels.shape, person_idxs.shape

((569, 320), (569,), (569,))

In [17]:
metrics = cross_val_score(data=statistics,
                          labels=labels,
                          person_idxs=person_idxs,
                        #   model=RandomForestClassifier(n_estimators=1000),
                          # model=LogisticRegression(C=100),
                          # model=SVC(C=1000),
                          model=KNeighborsClassifier(n_neighbors=50),
                          metric=accuracy_score,
                          n_test_persons=3,
                          random_state=0)
metrics

100%|██████████| 5/5 [00:00<00:00, 17.96it/s]

score [ 2 11  3]: 0.5172413793103449
score [10  0  4]: 0.5462184873949579
score [ 7  5 14]: 0.5263157894736842
score [12  6  9]: 0.5420560747663551
score [13  8  1]: 0.5575221238938053


,test_persons,train_metrics,test_metrics
0,"[2, 11, 3]",0.596026,0.517241
1,"[10, 0, 4]",0.575556,0.546218
2,"[7, 5, 14]",0.578022,0.526316
3,"[12, 6, 9]",0.595238,0.542056
4,"[13, 8, 1]",0.565789,0.557522


In [18]:
def pca_preproc(x_train, x_test):
    pca = PCA(n_components=.5, svd_solver='full').fit(x_train)
    return pca.transform(x_train), pca.transform(x_test)

metrics = cross_val_score(data=statistics,
                          labels=labels,
                          person_idxs=person_idxs,
                          # model=RandomForestClassifier(n_estimators=1000),
                        #   model=LogisticRegression(C=100),
                          # model=SVC(C=1000),
                          model=KNeighborsClassifier(n_neighbors=50),
                          metric=accuracy_score,
                          n_test_persons=3,
                          random_state=0,
                          data_preprocessor=pca_preproc)
metrics

 40%|████      | 2/5 [00:00<00:00,  7.91it/s]

score [ 2 11  3]: 0.5431034482758621
score [10  0  4]: 0.5210084033613446


 60%|██████    | 3/5 [00:00<00:00,  8.29it/s]

score [ 7  5 14]: 0.39473684210526316
score [12  6  9]: 0.5327102803738317


100%|██████████| 5/5 [00:00<00:00,  7.12it/s]

score [13  8  1]: 0.4778761061946903


,test_persons,train_metrics,test_metrics
0,"[2, 11, 3]",0.589404,0.543103
1,"[10, 0, 4]",0.564444,0.521008
2,"[7, 5, 14]",0.602198,0.394737
3,"[12, 6, 9]",0.551948,0.532710
4,"[13, 8, 1]",0.576754,0.477876


In [10]:
rf.feature_importances_.shape, rf.feature_importances_[rf.feature_importances_ != 0].shape

((1920,), (645,))

In [11]:
[spectrum_transformer.columns[i] for i in np.flatnonzero(rf.feature_importances_)]

['original_mean_2',
 'original_std_0',
 'original_std_1',
 'original_std_2',
 'original_std_3',
 'original_std_4',
 'original_std_5',
 'original_std_6',
 'original_std_7',
 'original_std_8',
 'original_std_9',
 'original_std_10',
 'original_std_11',
 'original_std_12',
 'original_std_13',
 'original_std_14',
 'original_std_15',
 'original_std_16',
 'original_std_17',
 'original_std_18',
 'original_std_19',
 'original_std_20',
 'original_std_21',
 'original_std_22',
 'original_std_23',
 'original_std_24',
 'original_std_25',
 'original_std_26',
 'original_std_27',
 'original_std_28',
 'original_std_29',
 'original_std_30',
 'original_std_31',
 'original_median_0',
 'original_median_1',
 'original_median_2',
 'original_median_5',
 'original_median_6',
 'original_median_7',
 'original_median_8',
 'original_median_10',
 'original_median_11',
 'original_median_13',
 'original_median_15',
 'original_median_17',
 'original_median_18',
 'original_median_19',
 'original_median_20',
 'original_m